In [0]:
%run /HOTREMA/constants

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, StructType, StructField
import json
import urllib.parse
import http.client

In [0]:
"""
Atrenkami darbuotojai, kurių šiandien gimtadienis.
Vasario 29-ta diena nėra tikrinima.
"""

select = """
select  
K14_KODAS_4A, 
K14_VARDAS as Vardas,
K14_PAV as Pavarde,
K14_GIM as GimimoData,
K14_MOB  AS `AsmeninisTel`,
K14_TEL AS `DarbinisTelefonas`,
K14_DIRBA
from rivile.K14_ASM
where K14_DIRBA = 1 and month(K14_GIM) = MONTH(current_date()) and DAY(K14_GIM) = DAY(current_date())
"""

df = spark.sql(select)
# display(df)

In [0]:
def check_phone_number(private_number, work_number):
  """ Čia įvedemas siunčiamas tekstas ir pakeičiamas numeris į formatą +370********"""
  
  text = "Nuoširdžiai sveikiname Jus gimimo dienos proga. Tegul kiekvieną Jūsų dieną įprasmina šviesios mintys, gražūs žodžiai ir darbai. Jūsų – Hotrema."
  text_russian = 'От всей души поздравляем Вас с днем рождения! Пусть светлые мысли, красивые слова и дела делают каждый Ваш день значимым. Ваша компания - Хотрема'
  try:
    if isinstance(private_number, str):
      number = int(private_number)
    elif isinstance(work_number, str):
      number = int(work_number)
    else:
      return None, None
  except:
    return None, None
  
  if 860000000 < number <  870000000:
    return (37000000000 + (number - 800000000)), text
  elif 37060000000 < number < 37070000000:
    return number, text
  else:
    return number, text_russian 

In [0]:
def sms_siuntimas(number, message):
  url =  'savitarna.smsbiuras.lt'
  conn = http.client.HTTPSConnection(url)
  message = urllib.parse.quote(message)    
  params ={
    'key': dbutils.secrets.get(scope='azure_key_vault', key="SMS-key"),
    'uid': dbutils.secrets.get(scope='azure_key_vault', key="SMS-Uid"),
    'sender': dbutils.secrets.get(scope='azure_key_vault', key="SMS-Sender"),
  }    
  nuoroda = f"/api?uid={params['uid']}&apikey={params['key']}&message={message}&from={params['sender']}&to={number}"
  conn.request("GET", nuoroda)
  res = conn.getresponse()
  conn.close()

raw: /mnt/raw
cleansing:	/mnt/cleansing
landing:	/mnt/landing
trusted:	/mnt/trusted

/mnt/raw	|| Already mounted.
/mnt/landing	|| Already mounted.
/mnt/trusted	|| Already mounted.
/mnt/cleansing	|| Already mounted.

In [0]:
def main():
  klaidos = []
  sveikinimo_numeriai = ''
  for row in df.rdd.collect():
    number, message = check_phone_number(row['AsmeninisTel'], row['DarbinisTelefonas'])
    if number:
      sms_siuntimas(number, message)
      numeris = str(number)
      sveikinimo_numeriai += numeris + ','
    else: 
      klaidos.append(row)
      
  if sveikinimo_numeriai!= '':
    """Čia galima įvesti numerį jei yra noras patikrinti kas gavo žinutes. 
    SMS būtų išsiunčiamas kas kartą, kai išsiunčiami sveikinimai."""
    pass
#     sms_siuntimas(37060035944, sveikinimo_numeriai)
  
  print(klaidos)

In [0]:
if __name__ == "__main__":
  main()

[]